In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "LDL_Cholesterol_Levels"
cohort = "GSE34945"

# Input paths
in_trait_dir = "../../input/GEO/LDL_Cholesterol_Levels"
in_cohort_dir = "../../input/GEO/LDL_Cholesterol_Levels/GSE34945"

# Output paths
out_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/GSE34945.csv"
out_gene_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/gene_data/GSE34945.csv"
out_clinical_data_file = "../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE34945.csv"
json_path = "../../output/preprocess/LDL_Cholesterol_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Candidate SNPs association with APOC3"
!Series_summary	"ApoC-III is a proatherogenic protein associated with elevated triglycerides; its deficiency is associated with reduced atherosclerosis. Mixed dyslipidemia, characterized by elevated triglyceride and apoC-III levels and low HDL cholesterol level, with or without elevated LDL cholesterol, increases cardiovascular disease risk and is commonly treated with combined statin and fibrate therapy. We sought to identify single nucleotide polymorphisms (SNPs) associated with apoC-III level response to combination therapy with statins and fenofibric acid (FA) in individuals with mixed dyslipidemia. Participants in a multicenter, randomized, double-blind, active-controlled study examining response to FA alone and in combination with statin were genotyped for candidate SNPs. Association between genotyed SNPs and APOC3 response to therapy was conducted"
!Series_overall_design	"We sought to identify single n

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the summary, this dataset appears to be specifically about SNPs (Single Nucleotide Polymorphisms)
# related to apoC-III levels in response to therapy, rather than gene expression data.
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait data (LDL Cholesterol Levels)
# From the background information, we know this cohort is about mixed dyslipidemia which involves elevated triglycerides,
# low HDL cholesterol, and sometimes elevated LDL cholesterol.
# In the sample characteristics, we can see row 2 contains "percent change in apoc3 levels"
# Since this is related to lipid metabolism and potentially affects LDL cholesterol levels, we can use this.
trait_row = 2  # Row with percent change in apoc3 levels

# For age data
# There's no explicit age information in the sample characteristics dictionary
age_row = None

# For gender data
# There's no explicit gender information in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

# Convert trait (percent change in apoc3 levels) to continuous numeric value
def convert_trait(value):
    # Extract the numerical value after the colon and convert to float
    try:
        # The format appears to be "percent change in apoc3 levels: -24.87309645"
        if isinstance(value, str) and ":" in value:
            numeric_value = float(value.split(":")[1].strip())
            return numeric_value
        elif isinstance(value, (int, float)):
            return float(value)
        else:
            return None
    except (ValueError, IndexError):
        return None

# Define convert_age and convert_gender as None since these data aren't available
convert_age = None
convert_gender = None

# 3. Save Metadata
# Set is_trait_available based on whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Convert the sample characteristics dictionary to a DataFrame
    # Create a DataFrame from the sample characteristics dictionary
    sample_char_dict = {0: ['disease state: Mixed dyslipidemia'], 
                        1: ['tissue: peripheral blood'], 
                        2: ['percent change in apoc3 levels: 5.298013245', 'percent change in apoc3 levels: -47.59825328', 
                            'percent change in apoc3 levels: -35.94470046', 'percent change in apoc3 levels: -23.8372093', 
                            'percent change in apoc3 levels: -31.57894737', 'percent change in apoc3 levels: -20.83333333', 
                            'percent change in apoc3 levels: -41.66666667', 'percent change in apoc3 levels: -27.92792793', 
                            'percent change in apoc3 levels: -26.76056338', 'percent change in apoc3 levels: -32.11382114', 
                            'percent change in apoc3 levels: -24.06417112', 'percent change in apoc3 levels: -14.48275862', 
                            'percent change in apoc3 levels: -18.23899371', 'percent change in apoc3 levels: -35.31914894', 
                            'percent change in apoc3 levels: -29.77099237', 'percent change in apoc3 levels: -36.95652174', 
                            'percent change in apoc3 levels: -27.91666667', 'percent change in apoc3 levels: -8.02919708', 
                            'percent change in apoc3 levels: -27.81065089', 'percent change in apoc3 levels: -29.76190476', 
                            'percent change in apoc3 levels: -24.87309645', 'percent change in apoc3 levels: -29.8245614', 
                            'percent change in apoc3 levels: -53.27510917', 'percent change in apoc3 levels: -7.352941176', 
                            'percent change in apoc3 levels: -27.40384615', 'percent change in apoc3 levels: -26.9058296', 
                            'percent change in apoc3 levels: -39.92395437', 'percent change in apoc3 levels: -40.75829384', 
                            'percent change in apoc3 levels: -8.888888889', 'percent change in apoc3 levels: -6.640625'], 
                        3: ['treatment group: fenofibric acid', 'treatment group: fenofibric acid+statin', 
                            'treatment group: statin alone']}
    
    # Convert to DataFrame format that geo_select_clinical_features expects
    # The function expects a DataFrame where rows are feature types and columns are samples
    
    # First, determine how many samples we have based on the most populated feature
    max_samples = max(len(values) for values in sample_char_dict.values())
    
    # Create an empty DataFrame with samples as columns
    clinical_data = pd.DataFrame(index=range(len(sample_char_dict)), columns=range(max_samples))
    
    # Fill the DataFrame with the sample characteristics
    for row_idx, values in sample_char_dict.items():
        for col_idx, value in enumerate(values):
            if col_idx < max_samples:
                clinical_data.loc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to the specified path
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [5.298013245], 1: [-47.59825328], 2: [-35.94470046], 3: [-23.8372093], 4: [-31.57894737], 5: [-20.83333333], 6: [-41.66666667], 7: [-27.92792793], 8: [-26.76056338], 9: [-32.11382114], 10: [-24.06417112], 11: [-14.48275862], 12: [-18.23899371], 13: [-35.31914894], 14: [-29.77099237], 15: [-36.95652174], 16: [-27.91666667], 17: [-8.02919708], 18: [-27.81065089], 19: [-29.76190476], 20: [-24.87309645], 21: [-29.8245614], 22: [-53.27510917], 23: [-7.352941176], 24: [-27.40384615], 25: [-26.9058296], 26: [-39.92395437], 27: [-40.75829384], 28: [-8.888888889], 29: [-6.640625]}
Clinical data saved to ../../output/preprocess/LDL_Cholesterol_Levels/clinical_data/GSE34945.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Candidate SNPs association with APOC3"
Line 1: !Series_geo_accession	"GSE34945"
Line 2: !Series_status	"Public on Jan 11 2012"
Line 3: !Series_submission_date	"Jan 09 2012"
Line 4: !Series_last_update_date	"Apr 02 2012"
Line 5: !Series_pubmed_id	"22236405"
Line 6: !Series_summary	"ApoC-III is a proatherogenic protein associated with elevated triglycerides; its deficiency is associated with reduced atherosclerosis. Mixed dyslipidemia, characterized by elevated triglyceride and apoC-III levels and low HDL cholesterol level, with or without elevated LDL cholesterol, increases cardiovascular disease risk and is commonly treated with combined statin and fibrate therapy. We sought to identify single nucleotide polymorphisms (SNPs) associated with apoC-III level response to combination therapy with statins and fenofibric acid (FA) in individuals with mixed dyslipidemia. Participants in a multicenter, randomized, double-blind, active-co

### Step 4: Gene Identifier Review

In [5]:
# Based on biomedical knowledge, the identifiers in this dataset are SNP IDs (rsIDs), not gene symbols
# These need to be mapped to their corresponding genes to be useful for gene expression analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 469699 rows

Gene annotation preview (first few rows):
{'ID': ['rs2294212', 'rs10889353', 'rs603446', 'rs5128', 'rs326'], 'SPOT_ID': ['rs2294212', 'rs10889353', 'rs603446', 'rs5128', 'rs326'], 'ILMN Strand': ['BOT', 'TOP', 'BOT', 'TOP', 'TOP'], 'SNP': ['[G/C]', '[A/C]', '[T/C]', '[C/G]', '[A/G]'], 'AddressA_ID': ['10', '23', '33', '51', '54'], 'ASO A': ['ACTTCGTCAGTAACGGACGCTCCCGGGTCTCCCGGG', 'ACTTCGTCAGTAACGGACGCCTGAGCCACCTTATCTGTTAAAA', 'ACTTCGTCAGTAACGGACGCTTGGACATCCAATCAGTTAGGGT', 'ACTTCGTCAGTAACGGACAGATTGCAGGACCCAAGGAGCTC', 'ACTTCGTCAGTAACGGACGAACTAGCTTGGTTGCTGAACACCA'], 'ASO B': ['GAGTCGAGGTCATATCGTGCTCCCGGGTCTCCCGGC', 'GAGTCGAGGTCATATCGTGCCTGAGCCACCTTATCTGTTAAAC', 'GAGTCGAGGTCATATCGTGCTTGGACATCCAATCAGTTAGGGC', 'GAGTCGAGGTCATATCGTAGATTGCAGGACCCAAGGAGCTG', 'GAGTCGAGGTCATATCGTGAACTAGCTTGGTTGCTGAACACCG'], 'GenomeBuild': ['hg18', 'hg18', 'hg18', 'hg18', 'hg18'], 'Chr': [20.0, 1.0, 11.0, 11.0, 8.0], 'Position': [43973970.0, 62890783.0, 

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the genetic data, we can see it contains SNP genotypes (AA, AB, BB, NC)
# We need to convert these to numeric values before mapping to genes

# Convert genotypes to numeric values
def convert_genotype_to_numeric(genotype):
    if genotype == 'AA':
        return 0
    elif genotype == 'AB':
        return 1
    elif genotype == 'BB':
        return 2
    else:  # NC or any other value
        return float('nan')

# Make a copy of the genetic data and convert genotypes to numeric values
numeric_gene_data = gene_data.copy()
for col in numeric_gene_data.columns:
    numeric_gene_data[col] = numeric_gene_data[col].apply(convert_genotype_to_numeric)

print("\nGenetic data after numeric conversion (preview):")
print(preview_df(numeric_gene_data))

print("\nExamining the annotation data to find how SNPs map to genes...")

# Extract basic annotation columns
annotation_subset = gene_annotation[['ID', 'Chr', 'Position']]
print("\nAnnotation subset:")
print(preview_df(annotation_subset))

# Create a custom mapping for SNPs to APOC3 and related genes
print("\nCreating custom mapping for SNPs based on available information...")

# Create a mapping dictionary - in a real application this would use
# a genomic database to map coordinates to genes, but here we'll use a simplified approach
snp_gene_map = {}

# For SNPs on chromosome 11 near APOC3 (116208649-116223862), map to APOC3
# For any SNPs on chromosome 8 near LPL (19841058-19869050), map to LPL 
# For SNPs on chromosome 19 near APOE (45409039-45412650), map to APOE
# These are approximations based on genomic locations of lipid metabolism genes
for idx, row in annotation_subset.iterrows():
    snp_id = row['ID']
    chrom = row['Chr']
    pos = row['Position']
    
    # Map SNPs to genes based on their genomic locations
    # This is a simplified mapping and would require a more robust approach in practice
    if pd.notna(chrom) and pd.notna(pos):
        if chrom == 11 and 116180000 <= pos <= 116230000:
            snp_gene_map[snp_id] = "APOC3"
        elif chrom == 8 and 19800000 <= pos <= 19900000:
            snp_gene_map[snp_id] = "LPL"
        elif chrom == 19 and 45300000 <= pos <= 45500000:
            snp_gene_map[snp_id] = "APOE"
        elif chrom == 1 and 62700000 <= pos <= 63000000:
            snp_gene_map[snp_id] = "ANGPTL3"
        else:
            # For SNPs without a clear gene association, assign a placeholder value
            snp_gene_map[snp_id] = f"SNP_chr{int(chrom)}_{int(pos)}"
    else:
        snp_gene_map[snp_id] = f"Unknown_{snp_id}"

# Create a mapping DataFrame
mapping_df = pd.DataFrame({'ID': list(snp_gene_map.keys()), 
                          'Gene': list(snp_gene_map.values())})

# Filter to only include SNPs that are in our gene_data
mapping_df = mapping_df[mapping_df['ID'].isin(numeric_gene_data.index)]

print("\nCreated mapping for SNPs to genes:")
print(preview_df(mapping_df))

# Apply gene mapping to convert SNP genotypes to gene-based data
gene_data = apply_gene_mapping(numeric_gene_data, mapping_df)

# Preview the resulting gene-based data
print("\nGene data after mapping:")
print(preview_df(gene_data))

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene data saved to {out_gene_data_file}")



Genetic data after numeric conversion (preview):


{'GSM858200': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858201': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858202': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858203': [2.0, 2.0, 2.0, 2.0, 2.0], 'GSM858204': [1.0, 2.0, 2.0, 0.0, 2.0], 'GSM858205': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858206': [1.0, 1.0, 2.0, 2.0, 2.0], 'GSM858207': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858208': [1.0, 2.0, 2.0, 0.0, 2.0], 'GSM858209': [2.0, 1.0, 2.0, 0.0, 2.0], 'GSM858210': [1.0, 2.0, 2.0, 0.0, 2.0], 'GSM858211': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858212': [1.0, 2.0, 1.0, 1.0, 2.0], 'GSM858213': [2.0, 2.0, 2.0, 2.0, 2.0], 'GSM858214': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858215': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858216': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858217': [2.0, 2.0, 2.0, 2.0, 2.0], 'GSM858218': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858219': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858220': [2.0, 2.0, 1.0, 0.0, 2.0], 'GSM858221': [2.0, 2.0, 2.0, 0.0, 2.0], 'GSM858222': [2.0, 2.0, 2.0, 2.0, 2.0], 'GSM858223': [2.0, 2.0, 2.0, 1.0, 2.0], 'GSM858224': [2.0, 2.0, 1.0, 0.0, 2.0],


Created mapping for SNPs to genes:
{'ID': ['rs2294212', 'rs10889353', 'rs603446', 'rs5128', 'rs326'], 'Gene': ['Unknown_rs2294212', 'Unknown_rs10889353', 'Unknown_rs603446', 'Unknown_rs5128', 'Unknown_rs326']}

Gene data after mapping:
{'GSM858200': [2.0], 'GSM858201': [2.0], 'GSM858202': [2.0], 'GSM858203': [2.0], 'GSM858204': [2.0], 'GSM858205': [2.0], 'GSM858206': [2.0], 'GSM858207': [2.0], 'GSM858208': [2.0], 'GSM858209': [2.0], 'GSM858210': [2.0], 'GSM858211': [2.0], 'GSM858212': [2.0], 'GSM858213': [2.0], 'GSM858214': [2.0], 'GSM858215': [2.0], 'GSM858216': [2.0], 'GSM858217': [1.0], 'GSM858218': [2.0], 'GSM858219': [2.0], 'GSM858220': [2.0], 'GSM858221': [2.0], 'GSM858222': [2.0], 'GSM858223': [2.0], 'GSM858224': [2.0], 'GSM858225': [2.0], 'GSM858226': [2.0], 'GSM858227': [2.0], 'GSM858228': [2.0], 'GSM858229': [2.0], 'GSM858230': [2.0], 'GSM858231': [2.0], 'GSM858232': [2.0], 'GSM858233': [2.0], 'GSM858234': [2.0], 'GSM858235': [2.0], 'GSM858236': [2.0], 'GSM858237': [2.0], 'G

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # This is SNP genotype data, not gene expression data
    # We should keep the original SNP IDs and not attempt to normalize as gene symbols
    print(f"Current gene data shape: {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Set is_gene_available to False as this is SNP data, not gene expression data
    is_gene_available = False
    print(f"Gene expression data available: {is_gene_available}")
    print("Note: This dataset contains SNP genotypes (AA/AB/BB), not gene expression measurements.")
except Exception as e:
    print(f"Error processing gene data: {e}")
    is_gene_available = False

# 2. Load the previously processed clinical data
print("\nLoading previously processed clinical data...")
try:
    # Load the clinical data that was saved in step 2
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with {clinical_df.shape[0]} rows and {clinical_df.shape[1]} columns")
    
    # Check if we have valid trait information
    is_trait_available = not clinical_df.empty
    print(f"Trait information available: {is_trait_available}")
    
    # Preview clinical data
    print("\nClinical data preview:")
    print(preview_df(clinical_df))
    
    # Check for single value bias in the trait data
    if is_trait_available and len(clinical_df) > 0:
        unique_values = clinical_df.iloc[:,0].unique()
        print(f"Number of unique trait values: {len(unique_values)}")
        if len(unique_values) == 1:
            print("Warning: The trait data contains only a single unique value.")
            is_biased = True
        else:
            # We'll set this preliminarily, but will evaluate properly later
            is_biased = False
    else:
        is_biased = True
    
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False
    clinical_df = pd.DataFrame()
    is_biased = True

# We don't need to link clinical and genetic data as we've determined
# this dataset contains SNP data, not gene expression data suitable for our analysis

# Validate and save cohort information
print("\nPerforming final validation...")
note = "Dataset contains SNP genotypes (AA/AB/BB), not gene expression measurements suitable for LDL cholesterol gene expression analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=clinical_df if not clinical_df.empty else pd.DataFrame({trait: []}),
    note=note
)

# Data is not usable for our gene expression analysis
print(f"\nDataset usability for {trait} association studies: {is_usable}")
print(f"Reason: {note}")


Normalizing gene symbols...
Loaded gene data with 1 genes and 1219 samples
Current gene data shape: 1 genes and 1219 samples
Gene expression data available: False
Note: This dataset contains SNP genotypes (AA/AB/BB), not gene expression measurements.

Loading previously processed clinical data...
Loaded clinical data with 1 rows and 30 columns
Trait information available: True

Clinical data preview:
{'0': [5.298013245], '1': [-47.59825328], '2': [-35.94470046], '3': [-23.8372093], '4': [-31.57894737], '5': [-20.83333333], '6': [-41.66666667], '7': [-27.92792793], '8': [-26.76056338], '9': [-32.11382114], '10': [-24.06417112], '11': [-14.48275862], '12': [-18.23899371], '13': [-35.31914894], '14': [-29.77099237], '15': [-36.95652174], '16': [-27.91666667], '17': [-8.02919708], '18': [-27.81065089], '19': [-29.76190476], '20': [-24.87309645], '21': [-29.8245614], '22': [-53.27510917], '23': [-7.352941176], '24': [-27.40384615], '25': [-26.9058296], '26': [-39.92395437], '27': [-40.7582